In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from warnings import filterwarnings
filterwarnings('ignore')

import statsmodels.api as sm
from scipy import stats
import statistics
from scipy.stats import shapiro
from statsmodels.stats import weightstats as stests

### Large Sample Test

#### One Sample Z Test

#### 1. We need to determine if girls' score on average higher than 65 in the reading test.  We collected the data of 517 girls by using random sampling from a normally distributed population and recorded their marks. Set ⍺ value (significance level) to 0.05.

Consider the reading score for female students given in the CSV file `StudentsPerformance.csv`. 

In [2]:
df_student = pd.read_csv('StudentsPerformance.csv')
df_student.gender.value_counts()

gender
female    517
male      483
Name: count, dtype: int64

In [3]:
scores = df_student[df_student['gender']=='female']['reading score']
scores.head()

0    55
1    63
2    71
5    85
6    51
Name: reading score, dtype: int64

In [4]:
# Let's first perform shapiro wilk test to check normality
# Ho: Data is normal
# Ha: Data is not normal

test,p_val = shapiro(scores)

print("Test Statistic",test)
print("P-Value",p_val)

Test Statistic 0.9949188828468323
P-Value 0.08642977476119995


Since, p-value is more than 0.05, we fail to reject null hypothesis. Therefore, Data is normally distributed.

In [5]:
# Ho: mu <= 65
# Ha: mu > 65

In [6]:
z_val = np.abs(round(stats.norm.isf(q = 0.05),2))
print("Z-Value for 5% level of significance is",z_val)

Z-Value for 5% level of significance is 1.64


In [7]:
ztest,p_val = stests.ztest(x1 = scores, value = 65, alternative = 'larger')
print("Z Score",ztest)
print("P-Value",p_val)

Z Score 2.529410071375873
P-Value 0.005712722457410142


As p-value is less than 0.05, we reject null hypothesis and hence conclude we have enough evidence that girls score on average is more than 65 in reading test.

In [8]:
def z_cal(samp_mean,pop_mean,samp_std,n):
    z_score = ((samp_mean-pop_mean)/(samp_std/np.sqrt(n)))
    return z_score

In [9]:
ci = stats.norm.interval(0.95, loc  = np.mean(scores), scale = statistics.stdev(scores)/np.sqrt(517))
print("Confidence Interval",ci)
# statistics.stdev is used as it takes into accoun ddof as we need sample standard deviation.

Confidence Interval (65.33094545307333, 67.6090932316462)


The confidence interval also ranges from 65 to 67. Z-Score is more than 1.64. Hence, we have enough evidence that girls score on average is more than 65 in reading test.

#### 2. The manager of a packaging process at a protein powder manufacturing plant wants to determine if the protein powder packing process is in control. The correct amount of protein powder per box is 350 grams on an average. A sample of 80 boxes was drawn which gave a mean of 354.5 grams with a standard deviation of 15. At 5% level of significance, is there evidence to suggest that the weight is different from 350 grams.

The null and alternative hypothesis is:

H<sub>0</sub>: $\mu = 350$<br>
H<sub>1</sub>: $\mu \neq 350$

Here ⍺ = 0.05, for a two-tailed test calculate the critical z-value.

In [10]:
# Ho: mu = 350
# H1: mu <> 350

In [11]:
z_val = np.abs(round(stats.norm.isf(q=0.05/2),2))
print("Critical value for 5% level of significance for two-tailed test",z_val)

Critical value for 5% level of significance for two-tailed test 1.96


In [12]:
samp_mean = 354.5
pop_mean = 350
n = 80
samp_std = 15

In [13]:
z_score = z_cal(samp_mean,pop_mean,samp_std,n)
print("Z-Score",z_score)
p_val = stats.norm.sf(z_score)
p_val_2 = p_val*2
print("P-Value",p_val)

Z-Score 2.6832815729997477
P-Value 0.003645179045767819


In [14]:
ci = stats.norm.interval(0.05, loc = samp_mean, scale = samp_std/np.sqrt(n))
print("Confidence interval",ci)

Confidence interval (354.3948375364017, 354.6051624635983)


Since P-value is less than 0.05, we reject null hypothesis and conclude that average weight of protein powder is different from 350gm.

### Two Sample Z Test

#### 1. The training institute Nature Learning claims that the students trained in their institute have overall better performance than the students trained in their competitor institute Speak Global Learning. We have a sample data of 500 students from each institute along with their total score collected from independent normal populations. Frame a hypothesis and test the Nature Learning's claim with 99% confidence.

Consider the total score for students given in the CSV file `StudentsPerformance.csv`. 

In [15]:
df_student = pd.read_csv("StudentsPerformance.csv")
df_student.head(2)

,gender,race/ethnicity,lunch,test preparation course,math score,reading score,writing score,total score,training institute
0,female,group B,standard,none,89,55,56,200,Nature Learning
1,female,group C,standard,completed,55,63,72,190,Nature Learning


In [16]:
nl_scores = df_student[df_student["training institute"]=="Nature Learning"]['total score']
n1 = len(nl_scores)
nl_mean = np.mean(nl_scores)
nl_std = statistics.stdev(nl_scores)

sgl_scores = df_student[df_student["training institute"]=="Speak Global Learning"]["total score"]
n2 = len(sgl_scores)
sgl_mean = np.mean(sgl_scores)
sgl_std = statistics.stdev(sgl_scores)

In [17]:
# Performing Shapiro Wilk test to Check for Normality

# Ho: Data is Normal
# H1: Data is not Normal

stat,p_val = stats.shapiro(df_student['total score'])
print("Test Statistic Value is",stat)
print("P-Value is",p_val)

Test Statistic Value is 0.9987770915031433
P-Value is 0.7399389743804932


Data is normally distributed as the p-value is greater than 0.05.

In [18]:
# Performing Levene's test to check for equality of variance

# Ho: There is equality of variance
# H1: There is no equality of variance

stat, p_val = stats.levene(nl_scores, sgl_scores)
print("Test Statistic",stat)
print("P-Value",p_val)

Test Statistic 0.6422721347822817
P-Value 0.42307998325221574


Since, p-value is greater than 0.05, we fail to reject null hypothesis. Therefore, there is equality of variance.

In [19]:
# Ho: mu1 <= mu2
# H1: mu1 > mu2

In [20]:
# Level of significance is 0.01
z_val = np.abs(round(stats.norm.isf(q=0.01),2))
print("Critical value for one-tailed z-test is",z_val)

Critical value for one-tailed z-test is 2.33


In [21]:
zscore, p_val = stests.ztest(x1 = nl_scores, x2 = sgl_scores, value = 0, alternative = 'larger')
print("Z-Score",zscore)
print("P-Value",p_val)

Z-Score 0.15125511253949914
P-Value 0.43988723840898025


In [22]:
ci = stats.norm.interval(0.99, loc = nl_mean-sgl_mean, scale = np.sqrt( ((nl_std**2)/n1) +((sgl_std**2)/n2) ))
print("Confidence Interval",ci)

Confidence Interval (-3.5585935666371427, 4.002593566637103)


Since p-value is more than 0.01 and the confidence interval contains value in null hypothesis (i.e. 0), we fail to reject null hypothesis. Therefore, we can conclude that there is no enough evidence for students trained by Nature learning perform better than Speak Global Learning. 

#### 2. A study was carried out to understand amount of haemoglobin in blood for males and females. A random sample of 160 males and 180 females have means of 13 g/dl and 15 g/dl. The two samples have standard deviation of 4.1 g/dl for male donors and 3.5 g/dl for female donor . Can it be said the population means of concentrations of the elements are the same for men and women? Use  α = 0.01.

In [23]:
# Ho: mu1-mu2 = 0
# H1: mu1-mu2 <> 0

In [24]:
z_val = np.abs(round(stats.norm.isf(q = 0.01/2),2))
print("Z-Value for 1% level of significance is",z_val)

Z-Value for 1% level of significance is 2.58


In [25]:
def twosampztest(samp_mean_1, samp_mean_2, samp_std_1, samp_std_2, value, n1, n2):
    denominator = np.sqrt((samp_std_1**2/n1) + (samp_std_2**2/n2))
    zscore = ((samp_mean_1 - samp_mean_2) - (value))/denominator
    return zscore

# value = pop_mean_1 - pop_mean_2 -- Here, this value is 0 (i.e. mu1-mu2 = 0)

In [26]:
sm_1 = 13
sm_2 = 15
n1 = 160
n2 = 180
sstd_1 = 4.1
sstd_2 = 3.5
value = 0

zscore = twosampztest(samp_mean_1 = sm_1, samp_mean_2 = sm_2, samp_std_1 = sstd_1, samp_std_2 = sstd_2, value = value, n1 = n1, n2 = n2)
print('Z-Score',zscore)

Z-Score -4.806830552525058


In [27]:
p_value = stats.norm.cdf(zscore)
req_p_val = p_value*2
print('P-value:',req_p_val)

P-value: 1.5334185117556497e-06


In [28]:
ci = stats.norm.interval(0.99, loc = sm_1-sm_2, scale = np.sqrt((sstd_1**2/n1) + (sstd_2**2/n2)))
print("Confidence Interval",ci)

Confidence Interval (-3.0717370938718864, -0.9282629061281136)


Since p-value is less than 0.01, we reject null hypothesis. Therefore we don't have enough evidence to conclude that both the population mean of men and women haemoglobin levels are equal.

### Small Sample Test

#### One Sample t Test

#### 1. A survey claims that in a math test female students tend to score fewer marks than the average marks of 75 out of 100. Consider a sample of 24 female students and perform a hypothesis test to check the claim with 90% confidence.

Dataset available in the CSV file `mathscore_1ttest.csv`.

In [29]:
df_scores = pd.read_csv('mathscore_1ttest.csv')
df_scores.head(2)

,gender,race/ethnicity,lunch,test preparation course,math score,reading score,writing score,total score,training institute
0,female,group C,standard,none,60,72,74,206,Nature Learning
1,female,group C,standard,none,59,72,68,199,Nature Learning


In [30]:
math_marks = list(df_scores['math score'])
sample_avg = np.mean(math_marks)
sample_std = statistics.stdev(math_marks)
n = len(math_marks)
dof = n-1
print("Degrees of freedom",dof)

Degrees of freedom 23


In [31]:
# Test of normality
stat,p_val = shapiro(math_marks)
print("Test Statistic",stat)
print("P-Value",p_val)

Test Statistic 0.9368310570716858
P-Value 0.13859796524047852


In [32]:
# Ho: mu>=75
# H1: mu<75

In [33]:
t_val = round(stats.t.isf(q = 0.1, df = 23),2)
print("Critical value for one tailed t-test",t_val)

Critical value for one tailed t-test 1.32


In [34]:
t_stat,p_val = stats.ttest_1samp(a = math_marks, popmean = 75)
req_p_val = p_val/2

print("Test Statistic",t_stat)
print("P-Value",req_p_val)

Test Statistic -3.6067380757023204
P-Value 0.0007426613957678669


In [35]:
ci = stats.t.interval(0.90, df = n-1, loc = sample_avg, scale = sample_std/np.sqrt(n))
print("Confidence Interval",ci)

Confidence Interval (62.399452707061236, 70.51721395960543)


Since p-value is less than 0.1, we reject null hypothesis. Therefore, we conclude that female students tend to score marks greater than 75 in the math test.

#### 2. A researcher is studying the growth of bacteria in waters of Lake Beach. The mean bacteria count of 100 per unit volume of water is within the safety level. The researcher collected 10 water samples of unit volume and found the mean bacteria count to be 94.8 with a sample variance of 72.66. Does the data indicate that the bacteria count is within the safety level? Test at the α = .05 level. Assume that the measurements constitute a sample from a normal population.

In [36]:
# Ho: mu>=100
# H1: mu<100

In [37]:
t_val = round(stats.t.isf(q = 0.05, df = 9),2)
print("Critical value for one-tailed t-test",t_val)

Critical value for one-tailed t-test 1.83


In [38]:
def ttest(pop_mean, samp_std, n, samp_mean):
    t_score = (samp_mean-pop_mean)/(samp_std/np.sqrt(n))
    return t_score

n = 10
pop_mean = 100
samp_var = 72.66
samp_mean = 94.8

samp_std = np.sqrt(samp_var)

t_score = ttest(pop_mean, samp_std, n, samp_mean)
print("T-Score",t_score)

T-Score -1.9291040236750068


In [39]:
p_value = stats.t.cdf(t_score, df = 9)
print("P-Value",p_value)

P-Value 0.04289782134327503


Since p-value is less than 0.05, we reject null hypothesis we can conlcude that the average bacteria per unit volume is within safety levels.

#### Two Sample t Test (Unpaired)

#### 1. The teachers' association claims that the total score of the students who completed the test preparation course is different than the total score of the students who have not completed the course. The sample data consists of 15 students who completed the course and 18 students who have not completed the course. Test the association's claim with ⍺ = 0.05.

Consider the total score of the students who have/ have not completed the preparation course are given in the CSV file `totalmarks_2ttest.csv`.

In [40]:
df_test_prep = pd.read_csv('totalmarks_2ttest.csv')
df_test_prep.head(2)

,gender,race/ethnicity,lunch,test preparation course,math score,reading score,writing score,total score,training institute
0,male,group E,standard,completed,84,83,78,245,Speak Global Learning
1,male,group C,free/reduced,completed,79,77,75,231,Speak Global Learning


In [41]:
course_comp = df_test_prep[df_test_prep['test preparation course'] == 'completed']['total score']
n1 = len(course_comp)
samp_avg1 = np.mean(course_comp)
samp_std1 = statistics.stdev(course_comp)

course_incomp = df_test_prep[df_test_prep['test preparation course'] == 'none']['total score']
n2 = len(course_incomp)
samp_avg2 = np.mean(course_incomp)
samp_std2 = statistics.stdev(course_incomp)

dof = n1+n2-2
print("Degrees of freedom",dof)

Degrees of freedom 31


In [42]:
# Test for normality

# Ho: Data is normal
# H1: Data is not normal

stat,p_val = shapiro(df_test_prep['total score'])
print("Statistic value",stat)
print("P-Value",p_val)

Statistic value 0.9845389723777771
P-Value 0.9080861806869507


Since p-value is more than 0.05 we fail to reject null hypothesis and therefore data is normal

In [43]:
# Levene's test to test equality of variance
stat,p_val = stats.levene(course_comp,course_incomp)
print("Statistic value",stat)
print("P-Value",p_val)

Statistic value 0.045113770764648356
P-Value 0.8331854285659768


In [44]:
# Ho: mu1 - mu2 = 0
# H1: mu1 - mu2 <> 0

In [45]:
t_val = np.abs(round(stats.t.isf(q = 0.05/2, df = dof),2))
print('Critical value of two tailed test',t_val)

Critical value of two tailed test 2.04


In [46]:
t_stat,p_val = stats.ttest_ind(a = course_comp, b = course_incomp)
print("Test statistics",t_stat)
print("P-Value",p_val)

Test statistics 1.4385323319823262
P-Value 0.16030339806989594


In [47]:
# pooled standard deviation
s = np.sqrt((((n1-1)*samp_std1**2) + ((n2-1)*samp_std2**2)) / (n1 + n2 - 2))

ci = stats.t.interval(0.95, df = n1 + n2 - 2, loc = samp_avg1 - samp_avg2, scale = s * np.sqrt(1/n1 + 1/n2))
print('95% confidence interval for population mean is', ci)

95% confidence interval for population mean is (-7.25533827425955, 41.9886716075929)


Since the p-value is greater than 0.05, also the confidence interval contains the value in the null hypothesis (i.e. 0). Therefore, we fail to reject the null hypothesis and conclude that there is no difference in the total marks of the students who have completed the preparation course and who have not completed the preparation course.

### Paired t Test

#### 1. A training institute wants to check if their writing training program was effective or not. 17 students are selected to check the hypothesis. Consider 0.05 as the level of significance.

The writing scores before and after training are provided in the CSV file `WritingScores.csv`. 

In [48]:
df_score = pd.read_csv('WritingScores.csv')

In [49]:
diff_marks = df_score['score_after'] - df_score['score_before']
mean_diff = np.mean(diff_marks)
std_diff = statistics.stdev(diff_marks)
n = len(df_score)
dof = n-1

In [50]:
# Test for normality

# Ho: Data is normal
# Ha: Data is not normal

print('Score Before')
stat,p_val = shapiro(df_score['score_before'])
print("Test Statistic",stat)
print("P-Value",p_val)
print()
print('Score After')
stat,p_val = shapiro(df_score['score_after'])
print("Test Statistic",stat)
print("P-Value",p_val)

Score Before
Test Statistic 0.947382390499115
P-Value 0.41645893454551697

Score After
Test Statistic 0.9686525464057922
P-Value 0.7944169044494629


From the above, p-value is more than 0.05 therefore we fail to reject null hypothesis.

In [51]:
# Ho: mu = 0 (The training was not effective)
# Ha: mu <> 0 The training was effective)

In [52]:
t_val = np.abs(round(stats.t.isf(q=0.05/2, df = dof),2))
print("Critical value for two-tailed t-test is",t_val)

Critical value for two-tailed t-test is 2.12


In [53]:
t_stat,p_val = stats.ttest_rel(df_score['score_after'],df_score['score_before'])
print("Test Statistic",t_stat)
print("P-Value",p_val)

Test Statistic 1.4394882729049499
P-Value 0.16929012896279846


In [54]:
ci = stats.t.interval(0.95, df = n-1, loc = mean_diff, scale = std_diff/np.sqrt(n))
print('95% confidence interval is',ci)

95% confidence interval is (-3.0029069531297283, 15.708789306070905)


Since, p-value is more than 0.05, we fail to reject null hypothesis. Therefore, we don't have enough evidence to conclude that the training was effective.

#### 2. An energy drink distributor claims that a new advertisement poster, featuring a life-size picture of a well-known athlete, will increase the product sales in outlets by an average of 50 bottles in a week. For a random sample of 10 outlets, the following data was collected. Test that the null hypothesis that there the advertisement was effective in increasing sales. Test the hypothesis using critical region technique. Use α = 0.05.

Given data:

        sales_before = [33, 32, 38, 45, 37, 47, 48, 41, 45]
        sales_after = [42, 35, 31, 41, 37, 36, 49, 49, 48]

In [55]:
sales_before = [33, 32, 38, 45, 37, 47, 48, 41, 45]
sales_after = [42, 35, 31, 41, 37, 36, 49, 49, 48]

In [56]:
n = len(sales_before)
diff_sale = np.array(sales_after)-np.array(sales_before)

mean_diff = np.mean(diff_sale)
std_diff = statistics.stdev(diff_sale, xbar = mean_diff)

dof = n-1
print("Degrees of freedom",dof)

Degrees of freedom 8


In [57]:
# Shapiro test of data normality
# Ho: Data is normal
# Ha: Data is not normal

stat,p_val = shapiro(sales_before)
print("Sales Before")
print("Test Statistic",stat)
print("P-Value",p_val)
print()
stat,p_val = shapiro(sales_after)
print("Sales After")
print("Test Statistic",stat)
print("P-Value",p_val)

Sales Before
Test Statistic 0.9187210202217102
P-Value 0.3817565143108368

Sales After
Test Statistic 0.9118874669075012
P-Value 0.3293103277683258


As p-value is more than 0.05 we fail to reject null hypothesis and conclude that Data is normally distributed.

In [58]:
# Ho:  mu difference <= 0 (The advertisement was not effective)
# Ha: mu difference > 0 (The advertisement was effective)

In [59]:
t_val = round(stats.t.isf(q = 0.05, df = dof),2)
print("Critical value for t-statistic",t_val)

Critical value for t-statistic 1.86


In [60]:
t_stat,p_val = stats.ttest_rel(sales_after,sales_before)
print("Test Statistic",t_stat)
print("P-Value",p_val)

Test Statistic 0.10085458113185983
P-Value 0.9221477146925299


As p-value is more than 0.05 we fail to reject null hypothesis and therefore conclude that the advertisement wasn't effective in boosting the sales.

### Z Proportion Test

#### One Sample Test

#### 1. In previous years, people believed that at most 80% of male students score more than 50 marks out of 100 in Mathematics. Perform a test to check whether this percentage is more than 80. Consider the level of significance as 0.05.

Consider the sample of math scores of male students available in the CSV file `StudentsPerformance.csv`.

In [61]:
 df_student = pd.read_csv('StudentsPerformance.csv')

In [62]:
total_male = df_student[df_student['gender']=='male'].shape[0]
male_marks_gt_50 = df_student[(df_student['gender']=='male') & (df_student['math score']>50)].shape[0]
prop_samp  = male_marks_gt_50/total_male

In [63]:
# Ho: Proportion <= 80
# Ha: Proportion > 80

In [64]:
z_val = np.abs(round(stats.norm.isf(q = 0.05),2))
print("Critical Value for one-tailed Z Proportion test is",z_val)

Critical Value for one-tailed Z Proportion test is 1.64


In [65]:
hypo_prop = 0.8

In [66]:
z_prop = (prop_samp - hypo_prop)/np.sqrt( (hypo_prop*(1-hypo_prop)) / total_male)
print("Test Statistic", z_prop)

Test Statistic 4.163394160018601


In [67]:
p_val  = stats.norm.sf(z_prop)
print("P-Value is",p_val)

P-Value is 1.5677570141208797e-05


In [68]:
ci = stats.norm.interval(0.95, loc = prop_samp, scale = np.sqrt( (hypo_prop*(1-hypo_prop)) / total_male))
print("Confidence Interval is",ci)

Confidence Interval is (0.8401038178124423, 0.9114489772186136)


As p-value is less than 0.05, we reject null hypothesis and test statistic 4.16 is more than 1.64 indication alternate hypothesis being more probable. Thereby, we conclude that more than 80% of the male students score more than 50 out of 100. 

#### 2. From a sample of 361 business owners had gone into bankruptcy due to recession. On taking a survey, it was found that 105 of them had not consulted any professional for managing their finance before opening the business. Test the null hypothesis that at most 25% of all businesses had not consulted before opening the business. Test the claim using p-value technique. Use α = 0.05.

In [69]:
# Ho: P <= 0.25
# Ha: P > 0.25

In [70]:
n = 361
x = 105 # No. of business owners who did not consult before
p_samp = x/n
hypo_p = 0.25
z_prop = (p_samp-hypo_p)/np.sqrt((hypo_p*(1-hypo_p))/n)
print("Z-Proportion",z_prop)

Z-Proportion 1.7928245201151534


In [71]:
p_val = stats.norm.sf(z_prop)
print("P-Value",p_val)

P-Value 0.03650049373124949


As p-value is less than 0.05, we reject null hypothesis. Therfore, we conclude that atleast 25% of all businesses had not consulted before opening business.

#### Two Sample Test

#### 1. A team of nutritionists believes that each institute provides 'standard' lunch to an equal proportion of students. A sample of students from institutes <i>Nature Learning</i> and <i>Speak Global Learning</i> is given. Consider the null hypothesis as equality of proportion with 0.1 level of significance.

Consider the sample data available in the CSV file `StudentsPerformance.csv`.

In [72]:
df_student = pd.read_csv('StudentsPerformance.csv')
df_student.head()

,gender,race/ethnicity,lunch,test preparation course,math score,reading score,writing score,total score,training institute
0,female,group B,standard,none,89,55,56,200,Nature Learning
1,female,group C,standard,completed,55,63,72,190,Nature Learning
2,female,group B,standard,none,64,71,56,191,Nature Learning
3,male,group A,free/reduced,none,60,99,72,231,Nature Learning
4,male,group C,standard,none,75,66,51,192,Nature Learning


In [73]:
df_n1 = df_student[df_student['training institute'] == 'Nature Learning']
n1 = df_n1.shape[0]
lunch_std1 = df_n1[df_n1['lunch'] == 'standard'].shape[0]

df_n2 = df_student[df_student['training institute'] == 'Speak Global Learning']
n2 = df_n2.shape[0]
lunch_std2 = df_n2[df_n2['lunch'] == 'standard'].shape[0]

In [74]:
# Ho: P1 - P2 = 0

# Ha: P1 - P2 <> 0

In [75]:
z_val = np.abs(round(stats.norm.isf(q=0.1/2),2))
print('Critical value for two tailed Z Test is',z_val)

Critical value for two tailed Z Test is 1.64


In [76]:
z_prop,p_val = sm.stats.proportions_ztest(count = np.array([lunch_std1,lunch_std2]),
                                         nobs = np.array([n1,n2]))

print('Test Statistic',z_prop)
print('p-Value',p_val)

Test Statistic 0.7935300106078008
p-Value 0.4274690915859791


In [77]:
p1 = lunch_std1/n1
p2 = lunch_std2/n2

# proportion of pooled sample
p_bar = (n1*p1 + n2*p2)/(n1+n2)

ci = stats.norm.interval(0.9, loc = p1-p2, scale = np.sqrt(p_bar*(1-p_bar)*(1/n1+1/n2)))
print("Confidence Interval for population proportion is",ci)

Confidence Interval for population proportion is (-0.02574794465126596, 0.07374794465126598)


As p-value is more than 0.10, we fail to reject null hypothesis and therefore we can conclude that there is not enough evidence to say that the proportion of students who get standard lunch is different.

#### 2. Steve owns a kiosk where he sells two magazines - A and B in a month. He buys 100 copies of magazine A out of which 78 were sold and 70 copies of magazine B out of which 65 were sold. Is there enough evidence to say that magazine is B is more popular? Test the claim using p-value technique with α = 0.05.

In [78]:
# Ho: Prop A >= Prop B
# Ha: Prop A < Prop B

In [79]:
z_val = np.abs(round(stats.norm.isf(q = 0.05),2))
print('Critical value for right tailed test is',z_val)

Critical value for right tailed test is 1.64


In [80]:
tot_A = 100 
tot_B = 70
A = 78
B = 65

In [81]:
z_prop,p_val = sm.stats.proportions_ztest(count = np.array([A,B]),nobs = np.array([tot_A,tot_B]),alternative  = 'smaller')

As p-value is less than 0.05, we reject null hypothesis and conclude that there is enough evidence to say that magazine B is more popular.

### Non-parametric Tests

#### Wilcoxon Signed Rank Test

#### One-sample Test

#### 1. A company manufactures auto ancillaries. One of them is steel rods with median diameter 10cm. A sample of 10 rods randomly selected from the production process gives the following results. Test the hypothesis that the median of the population has reduced. Use α = 0.05

given data:

    diameter = [9.1, 10.1, 9.9, 9.9, 10, 9.8, 9.7, 9.8, 9.9, 8.6]

In [82]:
diameter = np.array([9.1, 10.1, 9.9, 9.9, 10, 9.8, 9.7, 9.8, 9.9, 8.6])

In [83]:
# Ho: M >= 10
# Ha: M < 10

In [84]:
M_0 = 10 # Hypothesis median
diff  = diameter - M_0
diff

array([-0.9,  0.1, -0.1, -0.1,  0. , -0.2, -0.3, -0.2, -0.1, -1.4])

In [85]:
test_stat,p_val = stats.wilcoxon(x = diff, alternative = 'less')
print("Test statistic",test_stat)
print("P-Value",p_val)

Test statistic 2.5
P-Value 0.008364861494636245


As p-value is less than 0.05 we reject null hypothesis and thereby conclude that the there is enough evidence that the Median of population has reduced. 

#### 2. The Sweet Life company that produces hand sanitizers states that the in sanitizers the median percentage of alcohol is 0.82. A sample of 6 bottles is selected to test the claim that the median percentage of alcohol is not 0.82. Use  α = 0.1

Given data:

        alc_per = [0.32, 0.43, 0.38, 0.35, 0.85, 0.79]

In [86]:
alc_per = np.array([0.32, 0.43, 0.38, 0.35, 0.85, 0.79])

In [87]:
# Ho: Median = 0.82
# Ha: Median <> 0.82

In [88]:
Median_hypo = 0.82
diff = alc_per-Median_hypo 

In [89]:
test_stat,p_val = stats.wilcoxon(x = diff, alternative = 'two-sided')
print("Test Statistic",test_stat)
print("P-Value",p_val)

Test Statistic 2.0
P-Value 0.09375


AS p-value is less than 0.1, we reject null hypothesis and conclude that there exist enough evidence for the Median percentage of alcohol in sanitizers of SweetLife company not being 0.82.

#### Two-sample Paired Test

#### 1. The weights (in kg) of five hens before and after a special diet of millets was given. Test the hypothesis that the new millet diet has increased the weight of the hens at a 5% level of significance.

        before_wt = [2.7, 1.1, 1.4, 0.9, 0.9] 
        after_wt = [1.3, 1.4, 1.1, 1.3, 1.9] 

In [90]:
before_wt = [2.7, 1.1, 1.4, 0.9, 0.9] 
after_wt = [1.3, 1.4, 1.1, 1.3, 1.9] 

In [91]:
# Ho: Mean_diet <= 0
# Ha: Mean_diet > 0  i.e. Mean_diet_before - Mean_diet_after < 0 => Mean_diet_before < Mean_diet_after

In [92]:
test_stat,p_val = stats.wilcoxon(x = before_wt, y = after_wt, alternative = 'greater')
print("Test Statistic",test_stat)
print('P-Value',p_val)

Test Statistic 6.5
P-Value 0.6875


As p-value is more than 0.05, we fail to reject null hypothesis and therefore we don't have enough evidence to conclude that new millet diet has increased the weight of hens.

### Wilcoxon Rank Sum Test

#### 1. The lengths of time in minutes spent in the operating room by 9 patients undergoing the same operating method. 4 patients are from hospital A and 5 are from hospital B. Based on given data can we conclude that for the sample operative method, patients in hospital B tend to be longer than the patients in hospital A. Use level of significance as 0.05
Given data:

    A = [32, 31, 33, 46]
    B = [49, 20, 58, 55, 52]

In [93]:
A = [32, 31, 33, 46]
B = [49, 20, 58, 55, 52]

In [94]:
# Ho: Median A >= Median B
# Ha: Median A < Median B

In [95]:
z_val  = np.abs(round(stats.norm.isf(q = 0.05),2))
print('Critical value for one-tailed test is',z_val)

Critical value for one-tailed test is 1.64


In [96]:
test_stat,p_val = stats.ranksums(A,B)
req_p_val = p_val/2
print("Test statistic",test_stat)
print("P-Value",req_p_val)

Test statistic -1.4696938456699067
P-Value 0.0708223451475684


Since p-value is more than 0.05, we fail to reject null hypothesis. Therefore, we don't have enough evidence to conclude that hospital B patients have longer time spent in opertaing room than hospital A during the operative process.  

### Mann-Whitney U Test

#### 1. Two companies EyeCare and VisionFirst produces timolol eye drops. The sample of 5 bottles from both companies is selceted and the content of timolol maleate in milligram is recorded. Perform Mann-Whitney U test to test whether the amount of timolol maleate is different for both the companies. Use level of significance as 0.05.

Given data:

        eyecare = [6.18, 6.45, 6.21, 8.68, 8.45]
        visionfirst = [5.8, 7.8, 6.2, 5.9, 6.2]

In [97]:
eyecare = [6.18, 6.45, 6.21, 8.68, 8.45]
visionfirst = [5.8, 7.8, 6.2, 5.9, 6.2]

In [98]:
# Ho: Data is normal
# Ha: Data is not normal
stat,p_val = shapiro(eyecare)
print("Shapiro Test for eyecare,P-Value:",p_val)
stat,p_val = shapiro(visionfirst)
print("Shapiro Test for visionfirst,P-Value:",p_val)

Shapiro Test for eyecare,P-Value: 0.04114498198032379
Shapiro Test for visionfirst,P-Value: 0.025034727528691292


As p-value is less than 0.05 in both the cases, we reject null hypothesis and thereby conclude data is not normal.

In [99]:
# Ho: F_eyecare = F_visionfirst
# Ha: F_eyecare <> F_visionfirst
test_stat,p_val = stats.mannwhitneyu(eyecare, visionfirst, alternative = 'two-sided')
print("Test Statistic",test_stat)
print("P-Value",p_val)

Test Statistic 20.0
P-Value 0.14245669739409875


As p-value is more than 0.05, we fail to reject null hypothesis and therefore conclude that the amount of timolol maleate is same in both the eyedrops.

#### 2. The lengths of time in minutes spent in the operating room by 9 patients undergoing the same operating method. 4 patients are from hospital A and 5 are from hospital B. Based on given data can we conclude that for the sample operative method, patients in hospital B tend to be longer than the patients in hospital A. Use level of significance as 0.05.
Given data:

    A = [32, 31, 33, 46]
    B = [49, 20, 58, 55, 52]

In [100]:
A = [32, 31, 33, 46]
B = [49, 20, 58, 55, 52]

In [101]:
# Ho: Data is normal
# Ha: Data is not normal
stat,p_val = shapiro(A)
print("Shapiro Test for A,P-Value:",p_val)
stat,p_val = shapiro(B)
print("Shapiro Test for B,P-Value:",p_val)

Shapiro Test for A,P-Value: 0.02860516868531704
Shapiro Test for B,P-Value: 0.03407316282391548


As p-value is less than 0.05, we reject null hypothesis and therefore data is not normally distributed for both A and B hospitals.

In [102]:
# Ho: F_A >= F_B
# Ha: F_A < F_B

In [103]:
test_stat,p_val = stats.mannwhitneyu(A,B, alternative = 'less')
print("Test Statistic",test_stat)
print("P-Value",p_val)

Test Statistic 4.0
P-Value 0.09523809523809523


As p-value is more than 0.05, we fail to reject null hypothesis. Therefore, we don't have enough evidence to conclude the time taken for operation in hospital B is more than hospital A. 